In [ ]:
# 1. 데이터 파일 읽기
import pandas as pd

# 로컬 JSON 파일 읽기
df = pd.read_json("./data/documents.jsonl", lines=True)
df.tail()


In [ ]:
# 2. 안쓰는 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# 1. 프롬프트 템플릿 정의
SCIENCE_5W1H_TEMPLATE = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
**1차 요약**: [문서의 길이가 100자 이내라면 원문 그대로 사용 또는 400자 내외 요약]
"""


# 2. 프롬프트 객체 생성
science_prompt = PromptTemplate(
    input_variables=["document"],
    template=SCIENCE_5W1H_TEMPLATE
)

# 3. LLM 모델 선택 (Ollama 또는 OpenAI)
# llm = Ollama(model="llama3.1:8b")  # 로컬 모델
llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 4. 출력 파서 (문자열)
output_parser = StrOutputParser()

# 5. LCEL 체인 구성
science_summary_chain = (
    science_prompt 
    | llm 
    | output_parser
)

In [ ]:
# 3. 1차요약, 2차 요약 체인

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

from langchain_core.output_parsers import BaseOutputParser

llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

class SummaryOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # "요약:" 라벨 제거
        if text.startswith("요약:"):
            return text[3:].strip()
        elif "요약:" in text:
            return text.split("요약:")[-1].strip()
        return text.strip()

first_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.
        
문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""

second_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다. 
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {first_summary}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""

first_prompt = PromptTemplate(
    input_variables=["document"],
    template=first_summary
)


second_prompt = PromptTemplate(
    input_variables=["document"],
    template=second_summary

)

# 체인 생성 - 1차 요약 후 바로 2차 요약으로 연결
two_step_chain = (
    #{"text": RunnablePassthrough()}
    first_prompt 
    | llm 
    | SummaryOutputParser()
    #| (lambda x: print(f"1차 요약: {x}") or x) 
    #| (lambda x: {"first_summary": x})
    #| second_prompt
    #| llm
    #| SummaryOutputParser()
)



In [ ]:
# 4. 문서 -> 질문 생성 체인(안쓰는 체인)

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

from langchain_core.output_parsers import BaseOutputParser

make_question = """
당신은 지식과 상식문제 출제 전문가입니다. 다음 문서를 읽고 내용의 육하원칙과 핵심 키워드를 파악하여 전체 문서 내용과 가장 알맞은 질문을 생성해주세요.

문서: {document}

다음 기준으로 질문을 만들어주세요:
- 평균 30 ~ 40자, 최대 70자 정도로 생성
- 전체 내용을 보고 무엇에 대한 문제일까를 파악
- 구체적이고 실용적인 정보 중심
- 문서의 핵심 내용을 다양한 각도로 접근

질문:
"""

# 2. 프롬프트 객체 생성
question_prompt = PromptTemplate(
    input_variables=["document"],
    template=make_question
)

# 4. 출력 파서 (문자열)
output_parser = StrOutputParser()

# 5. LCEL 체인 구성
question_chain = (
    question_prompt 
    | llm 
    | output_parser
)

In [ ]:
# 5.  핵심 함수

# 요약문 생성
def get_summary(doc_id: str, document: str):
    
    # 1. 문서 요약
    if len(document) > 100:
        summary_result = two_step_chain.invoke({"document": document})
    else:
        summary_result = document
    return summary_result

# 파일저장
def save_summary(df, level=2):

    if level == 0:
        df.to_csv("./data/summary_ori.csv", index=False)    
    elif level == 1:
        df.to_csv("./data/summary_one.csv", index=False)    
    else:
        df.to_csv("./data/summary_two.csv", index=False)      

In [ ]:
# test
for index, data in df.iterrows():
    if index == 20: break
    
    summary_result, result = get_summary(data.docid, data.content)
    
    print("*"*50)
    print("*"*50)
    print(summary_result)
    print(result)

In [ ]:
# 6. 메인

def main():

    doc_id = []
    content = []
    summary = []
    for index, data in df.iterrows():
        #if index == 5: break
        
        result = get_summary(data.docid, data.content)
        
        # 진행상활 프린트
        if index % 50 == 0:
            print(f"{index}")
            print("*"*50)
            print("*"*50)
            print(data.content)
            print(result)

        doc_id.append(data.docid)
        content.append(data.content)
        summary.append(result)


    data = {
        'docid': doc_id,
        'content': content,
        'summary': summary
    }

    save_summary(pd.DataFrame(data), 1)



main()

